<a href="https://colab.research.google.com/github/k-vamshi/ieee-cis/blob/master/ieee_cis_fraud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
files.upload()

In [0]:
!pip install -q kaggle

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [0]:
 !kaggle competitions download -c IEEE-Fraud-detection --force

In [0]:
!kaggle datasets list -s ieee-fraud-detection

In [0]:
!unzip train_transaction.csv.zip -d train
!unzip train_identity.csv.zip -d train
!unzip test_transaction.csv.zip -d test
!unzip test_identity.csv.zip -d test
!unzip sample_submission.csv.zip

In [0]:
import numpy as np
import pandas as pd

In [0]:
from sklearn import preprocessing
import xgboost as xgb

In [0]:
train_transaction = pd.read_csv('train/train_transaction.csv', index_col='TransactionID')
test_transaction = pd.read_csv('test/test_transaction.csv', index_col='TransactionID')

train_identity = pd.read_csv('train/train_identity.csv', index_col='TransactionID')
test_identity = pd.read_csv('test/test_identity.csv', index_col='TransactionID')

sample_submission = pd.read_csv('sample_submission.csv', index_col='TransactionID')

In [0]:
train = train_transaction.merge(train_identity, how='left', left_index=True, right_index=True)
test = test_transaction.merge(test_identity, how='left', left_index=True, right_index=True)

print(train.shape)
print(test.shape)

y_train = train['isFraud'].copy()

# Drop target, fill in NaNs
X_train = train.drop('isFraud', axis=1)
X_test = test.copy()
X_train = X_train.fillna(-999)
X_test = X_test.fillna(-999)

In [0]:
del train, test, train_transaction, train_identity, test_transaction, test_identity

In [0]:
# Label Encoding
for f in X_train.columns:
    if X_train[f].dtype=='object' or X_test[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(X_train[f].values) + list(X_test[f].values))
        X_train[f] = lbl.transform(list(X_train[f].values))
        X_test[f] = lbl.transform(list(X_test[f].values)) 

In [0]:
clf = xgb.XGBClassifier(n_estimators=500,
                        n_jobs=4,
                        max_depth=9,
                        learning_rate=0.05,
                        subsample=0.7,
                        colsample_bytree=0.7,
                        missing=-999, 
                        gamma = 0.1,
                        verbose = 1)

clf.fit(X_train, y_train)

In [0]:
sample_submission['isFraud'] = clf.predict_proba(X_test)[:,1]
sample_submission.to_csv('simple_xgboost.csv')